In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
def write_prediction_to_file(prediction):
    header = "id,nextvisit"
    with open("tmp.csv", "w") as f:
        print(header, file=f)
        for idx, p in enumerate(prediction, 1):
            print(f"{idx}, {p}", file=f)

In [3]:
def visits2weekdays(visits):
    return np.apply_along_axis(lambda x: (x - 1) % 7 + 1, 0, visits)

def visits2history(visits, history_size):
    history = np.zeros(history_size, dtype=int)
    history[visits-1] = 1
    return history

In [4]:
with open('./train.csv', 'r') as f:
    data = pd.read_csv(f)
    data.visits = data.visits.apply(lambda x: (np.fromstring(x, dtype=int, sep=" ") - 1) % 7)

In [5]:
data.head()

,id,visits
0,1,"[1, 6, 6, 1, 3, 3, 3, 5, 5, 0, 4, 6, 6, 5, 4, ..."
1,2,"[2, 3, 4, 0, 3, 2, 4, 0, 2, 6, 5, 3, 4, 4, 2, ..."
2,3,"[1, 2, 4, 4, 4, 0, 6, 5, 5, 6, 4, 6, 5, 1, 6, ..."
3,4,"[0, 5, 0, 0, 4, 1, 0, 5, 1, 4, 6, 4, 5, 4, 1, ..."
4,5,"[5, 1, 6, 0, 4, 1, 1, 1, 6, 1, 1, 1, 1, 4, 1, ..."


### LSTM in Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Embedding
from keras.utils import to_categorical

# one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

In [ ]:
train_x = [row[-25:-1] for row in train_df.weekdays] 
train_y = [row[-1] for row in train_df.weekdays]
train_x = np.stack(train_x, axis=0)-1
train_y = np.stack(train_y, axis=0)-1

In [ ]:
train_x[0], train_y[0]

In [ ]:
#train_x = to_categorical(train_x, num_classes=7)
train_y = to_categorical(train_y, num_classes=7)

In [ ]:
model = Sequential()
model.add(Embedding(7, 16, input_length=24))
model.add(LSTM(100))
model.add(Dense(7, activation='softmax'))
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_x, train_y, batch_size=64, epochs=500, verbose=2)

In [ ]:
test_x = [row[-24:] for row in train_df.weekdays]
test_x = np.stack(test_x, axis=0)-1
test_y = np.array([row[-1] for row in train_df.weekdays])-1

pred = model.predict_classes(test_x ,verbose=2)

In [ ]:
pred += 1

In [ ]:
write_prediction_to_file(pred)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test_y, pred)

In [ ]:
test_x = np.stack(test_x, axis=0)
test_x.shape

In [ ]:
stats.mode(train_df.weekdays[0])

In [ ]:
DAYS_IN_WEEK = 7
WEEK_IN_MONTH = 4
DAYS_IN_MONTH = DAYS_IN_WEEK * WEEK_IN_MOTH
MONTH_IN_YEAR = 12